In [4]:
using Symbolics
using LinearAlgebra

Julia documention: https://docs.julialang.org/en/v1/


# Key generation

In [5]:
@variables x[1:36]
x[36]


# @variables a,b,c,t,x,y
# typeof(a+b)

# B=simplify.([t^2 + t + t^2  2t + 4t;
#                   x + y + y + 2t  x^2 - x^2 + y^2])
# B
expand(x[36]*(x[35]+x[33]))

x[33]*x[36] + x[35]*x[36]

In [6]:
function modify_coeffcients(A,q)
    
    coeffs_map=Symbolics.value(A).dict#turn the polynomial into matrix
    B=0#create our new polynomial
    for (key, value) in coeffs_map#iterate through the dict(monomial, coeffcients)
        if value>=q
            new_coef=value%q
            B+=new_coef*key
        else
            B+=key*value
        end  
    end
    B
end

modify_coeffcients (generic function with 1 method)

In [7]:
#t monomials
#bound the degree of mononials by b
#coefficients go from 1 to q-1(inclusive)
function sparse_polynomial(n,t,b,q)
    @variables x[1:n]
    
    sparse_poly=0
    
    for i in 1:t #we want t monomials
        deg=rand(1:b)
        
        monomial_d=1
        for j in 1:deg
            monomial_d*=x[rand(1:n)]
        end
        sparse_poly+=rand(1:(q-1))*monomial_d
        
    end
#     println(sparse_poly)
    A=modify_coeffcients(sparse_poly,q)
    return A
end

A=sparse_polynomial(4,5,3,6)
A

5(x[1]^2) + x[1]*x[2] + 2x[4] + 2(x[1]^2)*x[2] + x[2]

# matrix utility functions

In [41]:
A1 = [1 2 3; 4 1 6; 7 8 1]
det(A1)
I=diagm(0=>ones(3))#scalar*Identity matrix
I


3×3 Matrix{Float64}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

In [8]:
function num_identity(k)
    a=zeros(Num,k)
    a[1]=1
    for i in 2:k
        b=zeros(Num,k)
        b[i]=1
        a=[a b] 
    end
    return a
end
I_mat=num_identity(4)
I_mat[1,1]=x[2]*x[3]+x[1]
I_mat



4×4 Matrix{Num}:
 x[2]*x[3] + x[1]  0  0  0
                0  1  0  0
                0  0  1  0
                0  0  0  1

In [9]:
function generate_U(k,n,t,B,q)
    I_matrix=num_identity(k)#generate a k by k identity matrix
    for i in 1:Int((k*k-k)/2)
        b=rand(2:k)
        a=rand(1:(b-1))
        I_matrix[a,b]=sparse_polynomial(n,t,B,q)
    end

    return I_matrix
end 

U=generate_U(4,32,4,3,6)
U
# println(U)
# simplify.(expand.(inv(U)*U))


4×4 Matrix{Num}:
 1  …  3x[9] + 5x[17] + 4x[23] + 4x[9]*x[16]
 0                                         0
 0                                         0
 0                                         1

In [10]:
function generate_L(k,n,t,B,q)
    I_matrix=num_identity(k)#generate a k by k identity matrix
    for i in 1:Int((k*k-k)/2)
        a=rand(2:k)
        b=rand(1:a-1)
        I_matrix[a,b]=sparse_polynomial(n,t,B,q)
    end
    return I_matrix
end
L=generate_L(4,32,4,3,6)
expand.(simplify.(inv(L)*L))

# substitute.(inv(U)*U,(substitution_dict,))

4×4 Matrix{Num}:
 1.0  0.0  0.0  0.0
 0    1.0  0.0  0.0
 0    0    1.0  0.0
 0    0    0.0  1.0

In [12]:
A=[]
push!(A,1)
push!(A,'a')
println(size(A,1))
B=["kk",1]
vcat(A,B)
[i for i in 1:3]
zeros(Num,3,4)[:,2]


function cols_to_keep(k,l)
    col_opts=[i for i in 1:k]
    for i in 1:(k-l)
        random_number=size(col_opts,1)
        splice!(col_opts, rand(1:random_number))
    end
    return col_opts
end
cols_to_keep(4,3)

2


3-element Vector{Int64}:
 1
 2
 4

# utility functions

In [13]:
function random_del_cols(k,l,matrix)
#     col_opts=[i for i in 1:l]
    col_opts=cols_to_keep(k,l)
    matrix_kl=zeros(Num, k,l)
    
    j=1
    for i in col_opts
        matrix_kl[:,j]=matrix[:,i]
        j+=1
    end
    return col_opts,matrix_kl
end

random_del_cols(4,2,[5 6 6 8; 2 2 2 8; 6 6 2 8; 2 3 6 7])
# size([5 6 6 8; 2 2 2 8; 6 6 2 8; 2 3 6 7],2)
# size([5 6; 2 2; 6 6; 2 3])

([1, 4], Num[5 8; 2 8; 6 8; 2 7])

In [14]:
function find_kl_inverse(to_keep,S_inv)
    k=size(S_inv,1)#find the number of rows
    ml_inv=zeros(Num, length(to_keep),k)
    
    j=1
    for i in to_keep
        ml_inv[j,:]=S_inv[i,:]
        j+=1
    end
    return ml_inv
end
find_kl_inverse([1 2],[-17 -9 12 16;17 35/4 -47/4 -16;-4 -9/4 11/4 4; 1 3/4 -3/4 -1])
#*[5 6; 2 2; 6 6; 2 3]
    

2×4 Matrix{Num}:
 -17.0  -9.0    12.0    16.0
  17.0   8.75  -11.75  -16.0

In [15]:
1==1 && 4+5==9

true

In [16]:
function modify_neg_coeff(A,q)
    
    coeffs_map=Symbolics.value(A).dict#turn the polynomial into matrix
    B=0#create our new polynomial
    for (key, value) in coeffs_map#iterate through the dict(monomial, coeffcients)
        if value>=q || value<0 
            new_coef=value%6
            B+=new_coef*key
        else
            B+=key*value
        end  
    end
    B
end


modify_neg_coeff(-4(x[10]^2)*x[5]*x[25]*x[26] - 4x[10]*x[13]*x[25]*x[29] - 8x[4]*x[12]*x[21]*x[25]*x[28] - 5(x[1]^3)*(x[10]^2)*x[5]*x[8]*x[20]*x[26] - 5(x[1]^3)*x[8]*x[10]*x[13]*x[20]*x[29] - 10(x[1]^3)*x[4]*x[8]*x[12]*x[20]*x[21]*x[28] - (x[13]^2)*x[5]*x[10]*x[22]*x[23]*x[29]*x[30] - (x[5]^2)*(x[10]^2)*x[13]*x[22]*x[23]*x[26]*x[30] - 2x[4]*x[5]*x[12]*x[13]*x[21]*x[22]*x[23]*x[28]*x[30],6)

# modify_neg_coeff((x[10]^2)*x[5]*x[26] + x[10]*x[13]*x[29] + 2.0x[4]*x[12]*x[21]*x[28],6)


-4(x[10]^2)*x[5]*x[25]*x[26] - 4x[10]*x[13]*x[25]*x[29] - 2x[4]*x[12]*x[21]*x[25]*x[28] - 5(x[1]^3)*(x[10]^2)*x[5]*x[8]*x[20]*x[26] - 5(x[1]^3)*x[8]*x[10]*x[13]*x[20]*x[29] - 4(x[1]^3)*x[4]*x[8]*x[12]*x[20]*x[21]*x[28] - (x[13]^2)*x[5]*x[10]*x[22]*x[23]*x[29]*x[30] - (x[5]^2)*(x[10]^2)*x[13]*x[22]*x[23]*x[26]*x[30] - 2x[4]*x[5]*x[12]*x[13]*x[21]*x[22]*x[23]*x[28]*x[30]

In [17]:
Num(0)!=0.0
isone((x[10]^2)*x[5]*x[26] + x[10]*x[13]*x[29] + 2.0x[4]*x[12]*x[21]*x[28])
iszero((x[10]^2)*x[5]*x[26] + x[10]*x[13]*x[29] + 2.0x[4]*x[12]*x[21]*x[28])
isone(1.0)==true
Symbolics.value(1 + 9(x[9]^2)*x[1]*x[8]*x[23] + 9(x[10]^2)*x[1]*x[3]*x[4]*x[11]*x[17] + 9x[1]*x[3]*x[8]*x[9]*x[10]*x[11] + 9x[1]*x[4]*x[9]*x[10]*x[17]*x[23] + 15x[9]*x[12]*x[19]*x[23]*x[26]*x[30]*x[31] + 15x[1]*x[6]*x[8]*x[9]*x[18]*x[20]*x[22]*x[23] + 15x[3]*x[10]*x[11]*x[12]*x[19]*x[26]*x[30]*x[31] + 15x[1]*x[4]*x[6]*x[10]*x[17]*x[18]*x[20]*x[22]*x[23] + 25x[6]*x[12]*x[18]*x[19]*x[20]*x[22]*x[23]*x[26]*x[30]*x[31]).dict

Dict{Any, Any} with 9 entries:
  x[1]*x[6]*x[8]*x[9]*x[18]*x[20]*x[22]*x[23]                => 15
  x[3]*x[10]*x[11]*x[12]*x[19]*x[26]*x[30]*x[31]             => 15
  x[1]*x[3]*x[8]*x[9]*x[10]*x[11]                            => 9
  x[6]*x[12]*x[18]*x[19]*x[20]*x[22]*x[23]*x[26]*x[30]*x[31] => 25
  (x[10]^2)*x[1]*x[3]*x[4]*x[11]*x[17]                       => 9
  x[9]*x[12]*x[19]*x[23]*x[26]*x[30]*x[31]                   => 15
  (x[9]^2)*x[1]*x[8]*x[23]                                   => 9
  x[1]*x[4]*x[6]*x[10]*x[17]*x[18]*x[20]*x[22]*x[23]         => 15
  x[1]*x[4]*x[9]*x[10]*x[17]*x[23]                           => 9

In [18]:
function modify_matrix_coef(matrix,q)
    res=simplify.(expand.(matrix))#expand and then simplify
    
    for i in 1:size(res,1)
        for j in 1:size(res,2)
            a=res[i,j]

            if isone(a)==false && iszero(a)==false            
                res[i,j]=modify_neg_coeff(a,q)
            end
        end
    end
    return res
end

modify_matrix_coef([1.0 0 0 0.0; 0 1.0 0 0.0; 0 0 1 0.0; -4(x[10]^2)*x[5]*x[25]*x[26] - 4x[10]*x[13]*x[25]*x[29] - 8x[4]*x[12]*x[21]*x[25]*x[28] - 5(x[1]^3)*(x[10]^2)*x[5]*x[8]*x[20]*x[26] - 5(x[1]^3)*x[8]*x[10]*x[13]*x[20]*x[29] - 10(x[1]^3)*x[4]*x[8]*x[12]*x[20]*x[21]*x[28] - (x[13]^2)*x[5]*x[10]*x[22]*x[23]*x[29]*x[30] - (x[5]^2)*(x[10]^2)*x[13]*x[22]*x[23]*x[26]*x[30] - 2x[4]*x[5]*x[12]*x[13]*x[21]*x[22]*x[23]*x[28]*x[30] (x[10]^2)*x[5]*x[26] + x[10]*x[13]*x[29] + 2.0x[4]*x[12]*x[21]*x[28] 0 0.0],6)

4×4 Matrix{Num}:
                                                                                                                                                                                                                                                                                                                                                                                   1.0  …  0  0.0
                                                                                                                                                                                                                                                                                                                                                                                   0       0  0.0
                                                                                                                                                                                                                   

In [20]:
function matrix_kl(k,l,s,n,t,B,q)
    Ls=[]
    Us=[]
    for i in 1:s
        U=generate_U(k,n,t,B,q)
        L=generate_L(k,n,t,B,q)
        push!(Us,U)
        push!(Ls,L)
    end 
    U_new=Us[rand(1:s)]
    L_new=Ls[rand(1:s)]
    kk_matrix=L_new*U_new
    kk_matrix=modify_matrix_coef(kk_matrix,q)#modify the coefficients for kk_matrix
    kk_inv=inv(kk_matrix)#able to compute, not able to verify the result
    #kk_inv=inv(U_new)*inv(L_new)
    
    #make modifications to the coefficients of each entry in kk_matrix and kk_inv
#     kk_matrix=modify_matrix_coef(kk_matrix,q)
#     kk_inv=modify_matrix_coef(kk_inv,q)
    
    col_keep,kl_matrix=random_del_cols(k,l,kk_matrix)
    kl_inverse=find_kl_inverse(col_keep,kk_inv)
    
    #modify coefficients
#     kl_matrix=modify_matrix_coef(kl_matrix,q)
#     kl_inverse=modify_matrix_coef(kl_inverse,q)
    
    return kk_matrix,kk_inv,kl_matrix, kl_inverse
end 


A,B,C,D=matrix_kl(4,3,5,32,3,5,6)

println(size(C))
print(size(D))


(4, 3)
(3, 4)

In [239]:
@variables a,y,z
simplify(expand((a+y+z)^3))

a^3 + y^3 + z^3 + 3z*(a^2 + y^2) + 3a*(y^2 + z^2) + 3y*(a^2 + z^2) + 6a*y*z

In [21]:
res=D*C#please do inverse of kl matrix * kl matrix


3×3 Matrix{Num}:
 1 + (2(x[5] + x[14]) + 2x[8]*x[16]*x[19]*x[22]*x[26])*((-4x[6] - 2x[20] - 2x[10]*x[12]*x[20]) / ((-(5x[32] + 3x[4]*x[10] + 2x[13]*x[17]*x[21]*x[27] + 3(x[15]^2)*x[3]*x[10]*x[16]*x[19]*x[24]*x[27] + 4(x[3]^2)*x[6]*x[10]*x[14]*x[15]*x[19]*x[23]*x[32] + 4(x[26]^2)*x[3]*x[5]*x[6]*x[14]*x[17]*x[23]*x[28]*x[32] + 3x[3]*x[5]*x[6]*x[8]*x[10]*x[15]*x[18]*x[19]*x[22] + 2x[3]*x[6]*x[8]*x[9]*x[14]*x[16]*x[23]*x[26]*x[31]*x[32] - (3x[15]*x[16]*x[24]*x[27] + 2x[3]*x[6]*x[14]*x[23]*x[32] + 3x[5]*x[6]*x[8]*x[18]*x[22])*(5x[3]*x[10]*x[15]*x[19] + 2(x[26]^2)*x[5]*x[17]*x[28] + 4x[8]*x[9]*x[16]*x[26]*x[31] - (4x[6] + 2x[20] + 2x[10]*x[12]*x[20])*(3x[10]*x[13] + 3x[8]*x[12]*x[32] + 3x[19]*x[25]*x[28])))*(2(x[18]^2) + (x[13]^2)*x[7] + 2x[5]*x[12]*x[18] + 4x[13]*x[18]*x[30] + 2x[5]*x[6]*x[9]*x[21] + 2x[6]*x[9]*x[14]*x[21] + 2(x[18]^2)*x[10]*x[14]*x[30] + 3(x[21]^2)*x[20]*x[26]*x[31] + 2x[13]*x[18]*x[23]*x[31] + 4x[3]*x[5]*x[21]*x[30] + 4(x[13]^2)*x[23]*x[30]*x[31] + 4x[3]*x[14]*x[21]*x[30]

In [ ]:
res=simplify.(expand.(res))


# Substitution to verify

In [58]:
substitution_dict=Dict()
for i in 1:36
    substitution_dict[x[i]]=rand(1:10)
end 
substitution_dict

Dict{Any, Any} with 36 entries:
  x[29] => 9
  x[8]  => 5
  x[31] => 3
  x[36] => 1
  x[6]  => 7
  x[17] => 8
  x[28] => 4
  x[13] => 5
  x[33] => 8
  x[11] => 4
  x[16] => 9
  x[20] => 9
  x[26] => 6
  x[34] => 6
  x[30] => 4
  x[7]  => 6
  x[27] => 4
  x[21] => 10
  x[5]  => 7
  x[1]  => 10
  x[9]  => 8
  x[12] => 10
  x[3]  => 5
  x[4]  => 7
  x[18] => 8
  ⋮     => ⋮

In [71]:

substitute.(res,(substitution_dict,))

3×3 Matrix{Num}:
  1.0          0.0  0
 -1.69407e-21  1.0  0
  0.0          0.0  1.0